### Word In Visual Genome

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-1-Word In Visual Genome"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

#### Visual Genome Data

In [6]:
df_genome_objects = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Objects_Analysis.csv")
df_genome_objects.drop("image_url", inplace=True, axis=1)
df_genome_objects

,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,trees,1,tree.n.01,NaN,557,799,0,0
1,1058534,sidewalk,1,sidewalk.n.01,5046,290,722,78,308
2,1058508,building,1,building.n.01,NaN,538,222,1,0
3,1058539,street,1,street.n.01,3798578,258,359,439,283
4,1058543,wall,1,wall.n.01,NaN,535,135,0,1
...,...,...,...,...,...,...,...,...,...
2516934,3506171,number,2417997,numeral.n.01,NaN,24,28,188,228
2516935,3786799,plate,2417997,plate.n.04,NaN,54,89,238,233
2516936,3103226,road,2417997,road.n.01,NaN,83,90,312,247
2516937,3094589,road,2417997,road.n.01,NaN,121,495,2,208


In [7]:
df_genome_objects["names"].nunique()

82825

In [8]:
df_genome_attributes = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Attributes_Analysis.csv")
df_genome_attributes

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,clock,1,"green , tall",clock.n.01,339,79,421,91
1,5046,street,1,sidewalk,street.n.01,262,714,77,328
2,5045,shade,1,NaN,shade.n.01,192,274,119,338
3,1058529,man,1,NaN,man.n.01,262,60,238,249
4,5048,sneakers,1,grey,gym_shoe.n.01,26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,sky,2417997,"crystal clear , blue",sky.n.01,170,497,1,2
3802370,3103226,road,2417997,NaN,road.n.01,83,90,312,247
3802371,2712437,doors,2417997,NaN,door.n.01,144,50,98,127
3802372,3140197,bus,2417997,green,bus.n.01,155,225,29,109


In [9]:
df_genome_relationships = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Relationships_Analysis.csv")
df_genome_relationships

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,sidewalk,1058534,sidewalk.n.01,5046,290,722,78,308,15927,along.r.01,shade,5045,shade.n.01,192,274,119,338,on
1,1,shoes,1058525,shoe.n.01,5048,28,48,388,485,15928,wear.v.01,man,1058529,man.n.01,262,60,238,249,wears
2,1,NaN,5050,headlight.n.01,NaN,15,23,514,366,15929,have.v.01,car,5049,car.n.01,98,74,479,315,has
3,1,NaN,1058508,building.n.01,NaN,536,218,1,2,15930,along.r.01,sign,1058507,sign.n.02,182,88,118,13,on
4,1,NaN,1058534,sidewalk.n.01,NaN,266,722,77,331,15931,along.r.01,tree trunk,5055,trunk.n.01,327,87,622,234,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,NaN,3103225,bus.n.01,NaN,193,289,14,110,4245033,along.r.01,plate,3786799,plate.n.04,54,89,238,233,on
2316100,2417997,NaN,3786800,desert.n.01,NaN,119,157,301,211,4245034,in.r.01,NaN,3103225,bus.n.01,182,287,14,110,in
2316101,2417997,NaN,3103226,road.n.01,NaN,83,90,312,247,3722727,along.r.01,bus,3103225,bus.n.01,193,289,14,110,on
2316102,2417997,NaN,2712437,door.n.01,NaN,144,50,98,127,3880229,NaN,bus,3103225,bus.n.01,193,289,14,110,with


In [10]:
df_genome_question_answers = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Question_Answers_Analysis.csv")
df_genome_question_answers

,image_id,questions,que_and_anw_id,answers
0,2,what color is the car,986934,red
1,2,what color are the trees,986935,green
2,2,what has lots of glass windows,986936,the building
3,2,what is the guy doing,986937,walking
4,2,what is the man doing,986938,crossing the street
...,...,...,...,...
1445226,2417995,when was the photo taken,955110,day time
1445227,2417995,why is it so bright,955111,due to natural sunlight
1445228,2417995,how many trains are there,955112,one
1445229,2417995,what is the train made of,955113,steel


In [11]:
df_genome_region_descriptions = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-0-Visual Genome Process/Visual_Genome_Region_Descriptions_Analysis.csv")
df_genome_region_descriptions

,image_id,phrases,region_id,height,width,x_koor,y_koor
0,1,the clock is green in colour,1382,139,82,421,57
1,1,shade is along the street,1383,109,182,194,372
2,1,man is wearing sneakers,1384,30,61,241,491
3,1,cars headlights are off,1385,36,36,617,377
4,1,bikes are parked at the far edge,1386,49,41,322,298
...,...,...,...,...,...,...,...
5408684,2417997,a green school bus traveling down a road,5516956,181,303,0,111
5408685,2417997,numbers on the front of a bus,5516957,15,23,191,231
5408686,2417997,a yellow license plate on a bus,5516958,13,17,259,247
5408687,2417997,a country road between two mountains,5516959,112,317,180,218


#### Word Search In Genome Data

In [7]:
#df_word_raw = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/2-Word Select Process/{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
#df_word_raw

In [13]:
#word_list = ['an','anlama','doğum','gidiyor','günün','istediğim','istediğin','istediğiniz','kutlu','olmuş','soru','söylemek','zamanki','zor','şuna']

In [14]:
#df_word_raw = df_word_select[df_word_select["word"].isin(word_list)]
#df_word_raw.reset_index(drop=True, inplace=True)
#df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)
#df_word_raw

In [15]:
df_word_raw = pd.read_excel("Turkish_1000_Process.xlsx")
df_word_raw

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,NUM,NaN,bir,bir,bir,a,a,18835735
1,PRON,NaN,bu,bu,bu,this,this,11062659
2,PRON,Q,ne,ne,ne,what,what,8025880
3,CCONJ,NaN,ve,ve,ve,and,and,7766036
4,ADP,NaN,için,için,için,for,for,5484109
...,...,...,...,...,...,...,...,...
995,ADJ,NaN,resmi,resmi,resmi,formal,formal,68287
996,VERB,NaN,veriyor,ver,ver,giving,give,68163
997,NOUN,NaN,okul,okul,oku,school,school,68160
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [16]:
df_word_raw_word_trans_list = list(set(df_word_raw["word_en_translate"]))
df_word_raw_lemma_trans_list = list(set(df_word_raw["lemma_en_translate"]))

In [17]:
len(df_word_raw_word_trans_list)

818

In [18]:
len(df_word_raw_lemma_trans_list)

486

In [19]:
df_objects_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 50)
df_objects_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_objects_search_word_result

,word_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,in the middle,310063,in the middle,2411562,center.n.01,NaN,118,337,1,212
1,in the middle,1006958,man in the middle,2318811,"center.n.01 , man.n.01",NaN,110,46,280,116
2,in the middle,310060,in the middle area,2411562,area.n.01,NaN,124,332,5,164
3,in the middle,545355,grass in the middle,2380460,"center.n.01 , grass.n.01",NaN,28,500,0,236
4,dead,1206734,dead,2395246,dead.n.01,NaN,46,34,274,76
...,...,...,...,...,...,...,...,...,...,...
10483,drink,3914910,drink,2344444,NaN,3631335,80,115,317,161
10484,drink,3915790,drink,2344061,NaN,NaN,226,130,197,37
10485,drink,3915786,drink,2344061,NaN,NaN,79,90,31,64
10486,drink,2234787,drink,2343462,drink.n.01,"2461680 , 3405291",206,131,320,56


In [20]:
df_objects_search_word_result.to_csv("Visual_Genome_Objects_Word_Result.csv", index=False)

In [21]:
df_objects_search_lemma_result = word_group_dataframe(df_genome_objects, df_word_raw_lemma_trans_list, "names", 50)
df_objects_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_objects_search_lemma_result

,lemma_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,dead,1206734,dead,2395246,dead.n.01,NaN,46,34,274,76
1,dead,4453278,dead,2390706,NaN,NaN,110,499,0,223
2,dead,502291,dead,2389817,dead.n.01,NaN,46,101,88,305
3,dead,511101,dead,2387793,dead.n.01,NaN,127,110,195,177
4,dead,515179,dead,2387114,dead.n.01,515180,147,71,109,30
...,...,...,...,...,...,...,...,...,...,...
10018,drink,3914910,drink,2344444,NaN,3631335,80,115,317,161
10019,drink,3915790,drink,2344061,NaN,NaN,226,130,197,37
10020,drink,3915786,drink,2344061,NaN,NaN,79,90,31,64
10021,drink,2234787,drink,2343462,drink.n.01,"2461680 , 3405291",206,131,320,56


In [22]:
df_objects_search_lemma_result.to_csv("Visual_Genome_Objects_Lemma_Result.csv", index=False)

In [23]:
df_attributes_search_word_result = word_group_dataframe(df_genome_attributes, df_word_raw_word_trans_list, "attributes", 50)
df_attributes_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_attributes_search_word_result

,word_en_translate,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,in the middle,4360076,symbol,4022,in the middle,NaN,30,79,199,324
1,in the middle,1268755,traffic signal,2388459,in the middle,traffic_light.n.01,37,14,326,225
2,in the middle,515618,zebra,2387031,in the middle,zebra.n.01,165,240,199,58
3,in the middle,3845273,pillar,2386055,in the middle,pillar.n.01,90,26,243,127
4,in the middle,1315276,man,2383932,in the middle,man.n.01,151,127,189,34
...,...,...,...,...,...,...,...,...,...,...
12933,drink,3815430,can,2403964,energy drink,NaN,93,37,325,206
12934,drink,3307009,bottle,2360065,"lone , drink",bottle.n.01,60,20,393,269
12935,drink,1287774,glass,2386272,"tall , drink",glass.n.01,215,109,321,30
12936,drink,3917110,liquid,2343468,"amber , drink",NaN,34,36,256,317


In [24]:
df_attributes_search_word_result.to_csv("Visual_Genome_Attributes_Word_Result.csv", index=False)

In [25]:
df_attributes_search_lemma_result = word_group_dataframe(df_genome_attributes, df_word_raw_lemma_trans_list, "attributes", 50)
df_attributes_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_attributes_search_lemma_result

,lemma_en_translate,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,dead,1621451,ground,189,dead,land.n.04,113,789,9,66
1,dead,1846993,grass,2356699,dead,grass.n.01,66,349,55,228
2,dead,3134732,leaves,2356928,dead,leaf.n.01,8,13,307,384
3,dead,1779012,leaf,2356961,dead,leaf.n.01,50,104,260,284
4,dead,1979797,grass,2357003,dead,grass.n.01,47,367,62,172
...,...,...,...,...,...,...,...,...,...,...
11272,drink,3815430,can,2403964,energy drink,NaN,93,37,325,206
11273,drink,3307009,bottle,2360065,"lone , drink",bottle.n.01,60,20,393,269
11274,drink,1287774,glass,2386272,"tall , drink",glass.n.01,215,109,321,30
11275,drink,3917110,liquid,2343468,"amber , drink",NaN,34,36,256,317


In [26]:
df_attributes_search_lemma_result.to_csv("Visual_Genome_Attributes_Lemma_Result.csv", index=False)

In [27]:
df_relationships_search_word_result = word_group_dataframe(df_genome_relationships, df_word_raw_word_trans_list, "sub_name", 50)
df_relationships_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_relationships_search_word_result

,word_en_translate,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,in the middle,2411562,NaN,310064,flower.n.01,NaN,129,174,3,218,426714,lie.v.01,in the middle,310063,center.n.01,118,337,1,212,area lies a bush
1,in the middle,2318811,NaN,1006959,intersection.n.02,NaN,148,45,398,131,3007118,NaN,man in the middle,1006958,"center.n.01 , man.n.01",217,80,280,66,of
2,dead,2389817,NaN,502286,underbrush.n.01,NaN,55,247,3,123,1180507,leave.v.01,dead,502291,dead.n.01,46,112,1,148,leaves and
3,dead,2390706,NaN,4453277,NaN,NaN,76,483,14,257,4744971,NaN,dead,4453278,NaN,23,65,282,310,brown
4,dead,2390706,NaN,4453277,NaN,NaN,76,483,14,257,4744972,NaN,dead,4453278,NaN,23,65,282,310,brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8520,drink,2348369,NaN,1788229,glass.n.01,NaN,58,40,390,239,4419892,NaN,drink,3905829,NaN,42,30,395,252,in
8521,drink,2348661,NaN,2465745,glass.n.01,NaN,108,89,154,391,4418477,NaN,drink,3905104,NaN,37,65,158,409,in
8522,drink,2348674,NaN,1875432,glass.n.01,NaN,85,82,121,174,4418430,NaN,drink,3905084,NaN,80,73,124,175,in
8523,drink,2349442,computer desk,2568652,NaN,"2019307 , 2013738",110,500,0,261,3812269,sit.v.01,drink,2019306,drink.n.01,128,41,2,227,sitting on


In [28]:
df_relationships_search_word_result.to_csv("Visual_Genome_Relationships_Word_Result.csv", index=False)

In [29]:
df_relationships_search_lemma_result = word_group_dataframe(df_genome_relationships, df_word_raw_lemma_trans_list, "sub_name", 50)
df_relationships_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_relationships_search_lemma_result

,lemma_en_translate,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,dead,2389817,NaN,502286,underbrush.n.01,NaN,55,247,3,123,1180507,leave.v.01,dead,502291,dead.n.01,46,112,1,148,leaves and
1,dead,2390706,NaN,4453277,NaN,NaN,76,483,14,257,4744971,NaN,dead,4453278,NaN,23,65,282,310,brown
2,dead,2390706,NaN,4453277,NaN,NaN,76,483,14,257,4744972,NaN,dead,4453278,NaN,23,65,282,310,brown
3,dead,2387114,NaN,515154,field.n.01,NaN,168,429,60,114,1234013,NaN,dead,515179,dead.n.01,102,63,118,75,tree
4,dead,2387114,dead,515179,dead.n.01,515180,147,71,109,30,1234012,NaN,dead,515179,dead.n.01,102,63,118,75,tree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8353,drink,2348369,NaN,1788229,glass.n.01,NaN,58,40,390,239,4419892,NaN,drink,3905829,NaN,42,30,395,252,in
8354,drink,2348661,NaN,2465745,glass.n.01,NaN,108,89,154,391,4418477,NaN,drink,3905104,NaN,37,65,158,409,in
8355,drink,2348674,NaN,1875432,glass.n.01,NaN,85,82,121,174,4418430,NaN,drink,3905084,NaN,80,73,124,175,in
8356,drink,2349442,computer desk,2568652,NaN,"2019307 , 2013738",110,500,0,261,3812269,sit.v.01,drink,2019306,drink.n.01,128,41,2,227,sitting on


In [30]:
df_relationships_search_lemma_result.to_csv("Visual_Genome_Relationships_Lemma_Result.csv", index=False)

In [31]:
df_question_answers_search_word_result = word_group_dataframe(df_genome_question_answers, df_word_raw_word_trans_list, "questions", 50)
df_question_answers_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_question_answers_search_word_result

,word_en_translate,image_id,questions,que_and_anw_id,answers
0,in the middle,2364313,what in the middle,1719475,beans
1,in the middle,2332440,who is in the middle,337697,a baby elephant
2,in the middle,2391747,who is in the middle,1491577,a boy with a red shirt
3,in the middle,2377578,who is in the middle,365574,the man in the red tie
4,in the middle,2369526,who is in the middle,1825087,little boy
...,...,...,...,...,...
17987,to find,2370075,where would you look to find a reflection,1835712,the mirror
17988,to find,2409292,where can you go in arizona to find a ted s,182055,tempe
17989,to find,2398410,where would a person go to find public parking,1400501,to the left
17990,to find,2390416,what will these people hope to find in the water,1357514,waves


In [32]:
df_question_answers_search_word_result.to_csv("Visual_Genome_Question_Answers_Word_Result.csv", index=False)

In [33]:
df_question_answers_search_word_result2 = word_group_dataframe(df_genome_question_answers, df_word_raw_word_trans_list, "answers", 50)
df_question_answers_search_word_result2.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_question_answers_search_word_result2

,word_en_translate,image_id,questions,que_and_anw_id,answers
0,in the middle,2415083,where is the orange,135731,in the middle
1,in the middle,2373328,where is the big duck,1903693,in the middle
2,in the middle,2373769,where are the carrots,624441,in the middle
3,in the middle,2373917,where is the hot dog and pickle,1916072,in the middle
4,in the middle,2374351,where are the lines in the road,1924266,in the middle
...,...,...,...,...,...
17206,to find,2358095,why do the hats match,509096,to find the rented bike
17207,to find,2343787,what are the giraffes doing,776391,trying to find some shade
17208,to find,2347366,why is the lady picking food,773533,because she wants to find food to buy
17209,to find,2372158,why is the person walking away,1878746,perhaps to find a new place to skateboard


In [34]:
df_question_answers_search_word_result2.to_csv("Visual_Genome_Question_Answers_Word_Result2.csv", index=False)

In [35]:
df_question_answers_search_lemma_result = word_group_dataframe(df_genome_question_answers, df_word_raw_lemma_trans_list, "questions", 50)
df_question_answers_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_question_answers_search_lemma_result

,lemma_en_translate,image_id,questions,que_and_anw_id,answers
0,dead,2414032,what is dead,1022900,the grass
1,dead,2412069,what is dead,1116811,tree
2,dead,2326071,what is dead,726570,tree branch
3,dead,2410502,what is dead,1110687,leaves
4,dead,2354718,what is dead,383856,sheep
...,...,...,...,...,...
14207,drink,2356300,what drink is that,318609,coffee
14208,drink,2361403,where is the drink,250521,beside the plate
14209,drink,2360965,where is the drink,1652660,in the cup
14210,drink,2403483,where is the drink,1283744,bottom right


In [36]:
df_question_answers_search_lemma_result.to_csv("Visual_Genome_Question_Answers_Lemma_Result.csv", index=False)

In [37]:
df_question_answers_search_lemma_result2 = word_group_dataframe(df_genome_question_answers, df_word_raw_lemma_trans_list, "answers", 50)
df_question_answers_search_lemma_result2.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_question_answers_search_lemma_result2

,lemma_en_translate,image_id,questions,que_and_anw_id,answers
0,dead,61544,what is the condition of the limb,992581,dead
1,dead,2373611,what are the leaves on the ground,1909530,dead
2,dead,2373160,why is the grass brown,1900279,dead
3,dead,2403030,what condition is the grass,1273791,dead
4,dead,2372965,how are the leaves like,1896323,dead
...,...,...,...,...,...
13981,drink,2325424,what is the man holding in his right hand,855084,drink
13982,drink,61520,what is the man carrying,991564,drink
13983,drink,2372119,why is the can opened,1877783,drink
13984,drink,2327355,what is in the boy s left hand,533694,drink


In [38]:
df_question_answers_search_lemma_result2.to_csv("Visual_Genome_Question_Answers_Lemma_Result2.csv", index=False)

In [39]:
df_region_descriptions_search_word_result = word_group_dataframe(df_genome_region_descriptions, df_word_raw_word_trans_list, "phrases", 50)
df_region_descriptions_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_region_descriptions_search_word_result

,word_en_translate,image_id,phrases,region_id,height,width,x_koor,y_koor
0,in the middle,1824,man in the middle,5020001,92,75,370,220
1,in the middle,2335234,net in the middle,3695256,11,23,453,132
2,in the middle,2321022,bus in the middle,4730964,191,60,195,133
3,in the middle,2335054,man in the middle,3703882,132,120,186,112
4,in the middle,2403988,road in the middle,567874,150,250,6,182
...,...,...,...,...,...,...,...,...
22735,to find,2376277,cows are trying to find something to eat on gr...,1693716,39,102,63,246
22736,to find,2369988,snowfall on a field makes it hard to find gras...,2191100,85,493,2,288
22737,to find,2400473,oblong sign telling customers where to find mo...,733030,90,170,2,85
22738,to find,2348549,instructions where to find your life jacket if...,3214722,25,257,111,0


In [40]:
df_region_descriptions_search_word_result.to_csv("Visual_Genome_Region_Descriptions_Word_Result.csv", index=False)

In [41]:
df_region_descriptions_search_lemma_result = word_group_dataframe(df_genome_region_descriptions, df_word_raw_lemma_trans_list, "phrases", 50)
df_region_descriptions_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_region_descriptions_search_lemma_result

,lemma_en_translate,image_id,phrases,region_id,height,width,x_koor,y_koor
0,dead,2365024,dead bug,2427955,15,19,17,315
1,dead,2387126,dead wood,1773947,102,161,28,349
2,dead,2335986,dead leaf,3659889,28,43,243,412
3,dead,2342954,dead tree,3481126,72,62,428,158
4,dead,2394235,dead tree,1033141,267,116,12,0
...,...,...,...,...,...,...,...,...
18140,drink,2332443,two drink cans,3982934,70,450,23,0
18141,drink,2399986,a sports drink,756557,58,68,2,136
18142,drink,2360170,two drink taps,2659805,125,59,236,102
18143,drink,2379965,a drink cooler,1518899,41,50,382,250


In [42]:
df_region_descriptions_search_lemma_result.to_csv("Visual_Genome_Region_Descriptions_Lemma_Result.csv", index=False)

#### Copy Move And Delete

In [46]:
output_file = glob.glob(f"Visual_Genome_*_Result*.csv")
output_file

['Visual_Genome_Question_Answers_Word_Result2.csv',
 'Visual_Genome_Question_Answers_Lemma_Result2.csv']

In [47]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [48]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass